In [ ]:
import pandas as pd
from pathlib import Path

import numpy as np

In [ ]:
EXPERIMENTS_PATH = Path("../experiments")
results = list(Path(EXPERIMENTS_PATH).glob('**/output.txt'))

In [ ]:
dfs = []
for result in results:
    df = pd.read_csv(result, skiprows = 19, sep = "\t", header=None, names=['iter', 'train', 'test'])
    df['iter'] = df.iter.str.split("=").str[1].str.strip().astype(float)
    df['train'] = df.train.str.split("=").str[1].str.strip().astype(float)
    df['test'] = df.test.str.split("=").str[1].str.strip().astype(float)
    df['difference'] = df['test'] - df['train']
    df['experiment'] = result.parent.name
    dfs.append(df)
df_results = pd.concat(dfs)
df_results.to_csv("../results-mcmc.csv")

In [ ]:
idx = df_results.groupby(['experiment'])['test'].transform(min) == df_results['test']
df_results[idx]

,iter,train,test,difference,experiment
49,49.0,9.06689,9.53791,0.47102,metadata
49,49.0,9.01425,9.56320,0.54895,metadata_artist
49,49.0,8.02770,10.16130,2.13360,metadata_lyrics
49,49.0,8.47837,12.14040,3.66203,metadata_spotify
49,49.0,10.23070,10.24910,0.01840,triplets


In [ ]:
df_results.loc[df_results.iter==5]

,iter,train,test,difference,experiment
5,5.0,9.01440,9.63999,0.62559,metadata
5,5.0,9.02464,9.65549,0.63085,metadata_artist
5,5.0,8.88838,19.67900,10.79062,metadata_lyrics
5,5.0,29.08920,53.62220,24.53300,metadata_spotify
5,5.0,10.26750,10.26380,-0.00370,triplets


In [ ]:
df_melted = pd.melt(df_results, ['iter', 'experiment'], ['train', 'test'])
df_melted.head()

,iter,experiment,variable,value
0,0.0,metadata,train,9.77181
1,1.0,metadata,train,9.08557
2,2.0,metadata,train,9.01842
3,3.0,metadata,train,9.01413
4,4.0,metadata,train,9.01202


In [ ]:
import plotly.express as px

fig = px.line(df_results.loc[df_results.iter <= 10], x="iter", y="train", color="experiment",
              line_group="experiment", hover_name="experiment")
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(df_results.loc[df_results.iter <= 50], x="iter", y="test", color="experiment",
              line_group="experiment", hover_name="experiment")
fig.show()

In [ ]:
preds = list(Path(EXPERIMENTS_PATH).glob('**/output.libfm'))
tests = list(Path(EXPERIMENTS_PATH).glob('**/test.csv'))

In [ ]:
for i in range(len(preds)):
    print(tests[i])
    test_df = pd.read_csv(tests[i])
    pred_df = pd.read_csv(preds[i], header=None, names=['pred'])
    df_pred = pd.concat([test_df, pred_df], axis=1)

..\experiments\metadata\test.csv
..\experiments\metadata_artist\test.csv
..\experiments\metadata_lyrics\test.csv
..\experiments\metadata_spotify\test.csv
..\experiments\triplets\test.csv


In [ ]:
df_pred['actual_ranked'] = df_pred.groupby('user_id')['play_count'].rank(method="first")
df_pred['pred_ranked'] = df_pred.groupby('user_id')['pred'].rank(method="first")


In [ ]:
spearman = df_pred.groupby('user_id')['actual_ranked', 'pred_ranked'].corr(method='spearman').unstack().iloc[:,1]
np.mean(spearman)

c:\users\boon\anaconda3\envs\recommender\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



0.08075341483179907

In [ ]:
kendall = df_pred.groupby('user_id')['actual_ranked', 'pred_ranked'].corr(method='kendall').unstack().iloc[:,1]
np.mean(kendall)

c:\users\boon\anaconda3\envs\recommender\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



0.06808914095561586